## Tutorial on the one-class SVM enhanced by CNN features (using ResNet50) with heatmap visualization

## 1. Configurations

In [ ]:
from    typing      import  *
from    pathlib     import  Path

import  numpy   as  np
import  torch
from    sklearn.metrics     import  roc_auc_score, classification_report

from    utils       import  *
from    helper      import  *
from    models      import  OC_SVM

device = torch.device('cuda:2')

In [ ]:
# Set the root (Category: hazelnut)
base_path = Path(BASE_PATHS[MVTEC_AD])
category = 'bottle'
root = base_path/category

# Set the directories
train_normal_dir    = root / "train" / "good"
test_normal_dir     = root / "test" / "good"
test_anomaly_dirs   = [dir_subcat for dir_subcat in (root/"test").glob('*') if dir_subcat.name != "good"]

## 2. Load data

In [ ]:
# Load the train dataset
print(f"Loading the training dataset")
X_train = load_images(train_normal_dir, device=device)

# Load the test dataset
print(f"Loading the test dataset")
## Normal (0)
X_test  = load_images(test_normal_dir, device=device)
y_test  = np.zeros(len(X_test))
## Anomalous (1)
for anomaly_dir in test_anomaly_dirs:
    X_anom = load_images(anomaly_dir, device=device)
    X_test = torch.vstack([X_test, X_anom])
    y_test = np.hstack([y_test, np.ones(len(X_anom))])

In [ ]:
extractor = ConvFeatureMap(device)
extractor.eval()
with torch.inference_mode():
    feat_train = extractor.forward(X_train).mean((-2, -1)).cpu()
    feat_test  = extractor.forward(X_test).mean((-2, -1)).cpu()
    H, W = feat_train.shape[-2:]
    feat_train = feat_train.flatten(1).numpy()
    feat_test  = feat_test.flatten(1).numpy()
print("Train feature shape:", feat_train.shape)
print("Test feature shape: ", feat_test.shape)

## 3. Instantiate and train the model

In [ ]:
model = OC_SVM(kernel='rbf', gamma='scale', nu=0.05)
model.fit(feat_train)

## 5. Test the model

In [ ]:
# Decision_function
# * Positive value >>> inlier
# * Negative value >>> outlier
# ROC AUC without threshold
scores = model.decision_function(feat_test)
"""Higher value indicates higher probability of being predicted as an outlier (`1`); lower value indicates higher probability of being predicted as an inlier (`0`)."""
auc = roc_auc_score(y_test, scores)
print(f"ROC-AUC: {auc:.4f}")

# Setting the threshold (Example: 0)
y_pred = model.predict(feat_test)   # 1 for inliers, -1 for outliers
y_pred = np.where(y_pred==-1, 1, 0) # inlier: 1 -> 0 / outlier: -1 -> 1

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))

End of file